<a href="https://colab.research.google.com/github/OlgaWyz/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%reload_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:y_train = to_categorical(y_train)
if y_test.ndim == 1:y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%B%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

def predict(model_tarined, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 1.6369 - accuracy: 0.6657 - val_loss: 0.1780 - val_accuracy: 0.9616
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 0.2852 - accuracy: 0.9192 - val_loss: 0.0970 - val_accuracy: 0.9795
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1637 - accuracy: 0.9540 - val_loss: 0.0231 - val_accuracy: 0.9949
Epoch 4/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1068 - accuracy: 0.9719 - val_loss: 0.0311 - val_accuracy: 0.9949
Epoch 5/5
272/272 [==============================] - 4s 16ms/step - loss: 0.0964 - accuracy: 0.9727 - val_loss: 0.0237 - val_accuracy: 0.9929


0.9321995464852608

In [12]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.3137 - accuracy: 0.9322


[0.31366750597953796, 0.9321995377540588]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [23]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4576438831579421, 'dropout_cnn_block_three': 0.32464732673852253, 'dropout_cnn_block_two': 0.4666632446569522, 'dropout_dense_block_one': 0.5076577487963636, 'dropout_dense_block_two': 0.4705653496526514}
accuracy=0.9501133561134338
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4539524769656502, 'dropout_cnn_block_three': 0.34835063488854917, 'dropout_cnn_block_two': 0.3115653541629421, 'dropout_dense_block_one': 0.6850974811744208, 'dropout_dense_block_two': 0.5455886855616123}
accuracy=0.9496598839759827
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.3360756998454498, 'dropout_cnn_block_three': 0.4588567639882932, 'dropout_cnn_block_two': 0.3195011112926559, 'dropout_dense_block_one': 0.6118391595150972, 'dropout_dense_block_two': 0.5175639211890924}
accuracy=0.9462584853172302
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4612010933683765, 'dropout_cnn_block_three': 0.4690356707093435, 'dropout_cnn_block_two': 0.33120701617937